In [9]:
import matplotlib.pyplot as plt
import pandas as pd

from functions import *

from scipy.optimize import minimize
from multiprocess import Pool

In [10]:
#Set params

n = 400

#Data generation 
X = np.random.uniform(-2,2,n)
X.sort()

eps = np.random.normal(0, 1,  n)

target = lambda a: fan(X,n,a,eps)

#Missing values 

b0 = -0.7
b1 =  -1

a =1

y = target(a)

omega = bernoulli.rvs(pi(y,b0,b1), size=n)

df = pd.DataFrame({'x':X,'y':y,'omega':omega})

#ipw

df['y_pi'] =  df['y'].apply(lambda t: t/pi(t,b0,b1))

df['var'] = df['x'].apply(lambda t: sigma_reg(t,len(df)))

df_cc = df.loc[df['omega']==1].reset_index(drop=True)

In [11]:
def r_parallel_cv(h):
    #print(1)
    def fn(j):
        df1 = df.drop([j],axis=0)
        y_hat = nw(h,df.x[j],df1.x,df1.y)
        
        return (df.y[j]-y_hat)**2 #/pi(df.y[j],b0,b1)

    with Pool(8) as p:
        return np.mean(p.map(fn, range(len(df))))

In [12]:
h  = minimize(r_parallel_cv, 0.1).x

In [13]:
h

array([-0.07335405])